In [1]:
import os
import json
import time
import math
import matplotlib.pyplot as plt
from core.data_processor import DataLoader
from core.model import Model
import numpy as np

import pandas as pd
# import numpy as np


Using TensorFlow backend.


In [2]:
def convert_datetime_to_checkpoint(realdata, datetime):
    # 这是针对于有header的打他frame？是这样吗？需要验证
    checkpoint = realdata.loc[realdata['Date'] == datetime].index[0]
    return checkpoint 

def convert_checkpoint_to_datetime(realdata, checkpoint):
    datetime = realdata.iloc[checkpoint]['Date']
    return datetime 

2020-01-13 尝试进行对 x_test 进行改造，从而提高效率。

In [3]:
stock = "000738"
column = "High"
startDate = "2019-09-10"
sequence_length = "7"

configs = json.load(open('configs/config.json', 'r'))
if column == "Close" :
    configs['data']['columns'] = ["Close", "Open"]
elif column == "High" :
    configs['data']['columns'] = ["High",  "Open"]
elif column == "Low" :
    configs['data']['columns'] = ["Low", "Open"]

configs['data']['filename'] = stock + ".csv"
configs['data']['sequence_length'] = int(sequence_length)
configs['model']['layers'][0]['input_timesteps'] = int(sequence_length) - 1


model_save_dir = "bestmodels/" + sequence_length + "/" + stock + "-" + column + ".h5"
model = Model()
model.load_model(model_save_dir)

realdata = pd.read_csv('data/' + stock + '.CSV')
startPoint = convert_datetime_to_checkpoint(realdata, startDate)
data_windows = realdata.get(configs['data']['columns']).values[startPoint-configs['data']['sequence_length']:startPoint]

def normalise_windows(window_data, single_window=False):
    '''Normalise window with a base value of zero'''
    normalised_data = []
    window_data = [window_data] if single_window else window_data
    for window in window_data:
        normalised_window = []
        for col_i in range(window.shape[1]):
            normalised_col = [((float(p) / float(window[0, col_i])) - 1) for p in window[:, col_i]]
            normalised_window.append(normalised_col)
        normalised_window = np.array(normalised_window).T # reshape and transpose array back into original multidimensional format
        normalised_data.append(normalised_window)
    return np.array(normalised_data)


# print(data_windows)
data_windows = np.array(data_windows).astype(float)
data_windows = normalise_windows(data_windows, single_window=True)

predictions = model.predict_sequence_3(data_windows, 7)

price_list = []
for i in predictions :
    price_list.append(realdata.loc[startPoint-configs['data']['sequence_length'],column] * (1 + i))

print(predictions)



[-0.044081304, -0.04142143, -0.036856025]


In [3]:
stock = "000738"
column = "High"
startDate = "2019-05-07"
# endDate = "2019-08-27"

def predict(startDate, stock, column, sequence_length ):
    configs = json.load(open('configs/config.json', 'r'))
    model_save_dir = "bestmodels/" + sequence_length + "/" + stock + "-" + column + ".h5"
    
    if column == "Close" :
        configs['data']['columns'] = ["Close", "Open"]
    elif column == "High" :
        configs['data']['columns'] = ["High",  "Open"]
    elif column == "Low" :
        configs['data']['columns'] = ["Low", "Open"]

    configs['data']['filename'] = stock + ".csv"
    configs['data']['sequence_length'] = int(sequence_length)
    configs['model']['layers'][0]['input_timesteps'] = int(sequence_length) - 1
    

    data = DataLoader(
        os.path.join('data', stock + ".csv"),
        0,
        configs['data']['columns']
    )
    
    x_test, y_test = data.get_test_data(
        configs['data']['sequence_length'], 
        normalise=configs['data']['normalise']
    )
    model = Model()
    model.load_model(model_save_dir)
    
    realdata = pd.read_csv('data/' + stock + '.CSV')
    startPoint = convert_datetime_to_checkpoint(realdata, startDate)
    
    # 需要找到 i， 第 i 个需要预测的数据。
    prediction = model.predict_sequence_full(x_test[startPoint], configs['data']['sequence_length'])
    
    price_list = []
    for i in prediction :
        price_list.append(realdata.loc[startPoint,column] * (1 + i))

    return price_list[:3]


In [4]:
def give_me_5_predictions(startDate, stock, column):
    p1 = predict(startDate, stock, column, "3")
    p2 = predict(startDate, stock, column, "4")
    p3 = predict(startDate, stock, column, "5")
    p4 = predict(startDate, stock, column, "6")
    p5 = predict(startDate, stock, column, "7")
    
    five_predictions = [p1, p2, p3, p4,p5]
    predictions_frame = pd.DataFrame(seven_predictions) 
    
    realdata = pd.read_csv('data/' + stock + '.CSV')
    startPoint = convert_datetime_to_checkpoint(realdata, startDate)
    
#     print( realdata.loc[startPoint+1:startPoint+3, ["Date", column] ].values[:] )
    days = realdata.loc[startPoint+1:startPoint+3, ["Date", column] ].values[:]
    
    predictions = {
        "column":column,
        startDate:
        {
            days[0][0]:{"mean":predictions_frame[0].mean(),"mid":predictions_frame[0].median()},
            days[1][0]:{"mean":predictions_frame[1].mean(),"mid":predictions_frame[1].median()},
            days[2][0]:{"mean":predictions_frame[2].mean(),"mid":predictions_frame[2].median()}
        }
    }
    file_to_save = "predictions_result/" + stock + "_" + column + "_" + startDate + ".json"
    
    with open(file_to_save, 'w') as f:
        json.dump(predictions, f)
    
    return predictions
    
    

In [ ]:

startDate = "2019-01-18"
endDate = "2019-01-31"

realdata = pd.read_csv('data/' + stock + '.CSV')
startPoint = convert_datetime_to_checkpoint(realdata, startDate)
endPoint = convert_datetime_to_checkpoint(realdata, endDate)

days = realdata.loc[startPoint:endPoint, ["Date"] ].values

# print(days)
for day in days :
    
    give_me_5_predictions(day[0], stock, "High")
    print(day[0], " High finish")
    
    give_me_5_predictions(day[0], stock, "Low")
    print(day[0], " Low finish")
    
#     give_me_5_predictions(day[0], stock, "Close")
#     print(day[0], " CLose finish")
    
# column = "Low"
# give_me_5_predictions(startDate, stock, column)
# print("high")
# column = "High"
# give_me_5_predictions(startDate, stock, column)
# print("finish")

2019-01-03  High finish
2019-01-03  Low finish
2019-01-03  CLose finish
2019-01-04  High finish
2019-01-04  Low finish
2019-01-04  CLose finish
2019-01-07  High finish
2019-01-07  Low finish
2019-01-07  CLose finish
2019-01-08  High finish
2019-01-08  Low finish
2019-01-08  CLose finish
2019-01-09  High finish
2019-01-09  Low finish
2019-01-09  CLose finish
2019-01-10  High finish
2019-01-10  Low finish
2019-01-10  CLose finish
2019-01-11  High finish
2019-01-11  Low finish
2019-01-11  CLose finish
2019-01-14  High finish
2019-01-14  Low finish
2019-01-14  CLose finish
2019-01-15  High finish
2019-01-15  Low finish
2019-01-15  CLose finish
2019-01-16  High finish
2019-01-16  Low finish
2019-01-16  CLose finish
2019-01-17  High finish
2019-01-17  Low finish
2019-01-17  CLose finish
2019-01-18  High finish


In [ ]:
startDate = "2019-01-03"
endDate = "2019-01-31"

realdata = pd.read_csv('data/' + stock + '.CSV')
startPoint = convert_datetime_to_checkpoint(realdata, startDate)
endPoint = convert_datetime_to_checkpoint(realdata, endDate)

days = realdata.loc[startPoint:endPoint, ["Date"] ].values

# print(days)
for day in days :
    
    give_me_5_predictions(startDate, stock, "High")
    print(day[0], " High finish")
    
    give_me_5_predictions(startDate, stock, "Low")
    print(day[0], " Low finish")
    
    give_me_5_predictions(startDate, stock, "Close")
    print(day[0], " CLose finish")

In [5]:
startdate = "2019-08-23"
realdata = pd.read_csv('data/' + stock + '.CSV')
checkpoint = convert_datetime_to_checkpoint(realdata, startdate)
date = convert_checkpoint_to_datetime(realdata, checkpoint)
print(startdate)
print(date)

2019-08-23
2019-08-23


In [6]:
startDate = "2019-08-12"
column = "High"
give_me_5_predictions(startDate, stock, column)

real data
[['2019-08-13' 13.63]
 ['2019-08-14' 13.66]
 ['2019-08-15' 13.59]]
data frame
           0          1          2
0  13.452866  13.530969  13.642185
1  13.643313  13.676730  13.660977
2  13.666673  13.643024  13.586893
3  13.648509  13.685497  13.701070
4  13.740333  13.758827  13.752094
mean and median
13.630338668505662 13.648509107055142
13.659009544333443 13.676730103944427
13.66864367374149 13.66097725968808


In [11]:

realdata = pd.read_csv('data/' + stock + '.CSV')
column = "High"

startDate = "2019-05-06"


startPoint = convert_datetime_to_checkpoint(realdata, startDate)
print(startPoint)
print("realdata",realdata.loc[startPoint:startPoint+1,:] )

data = DataLoader(
    os.path.join('data', stock + ".csv"),
    0,
    ["Open", "High", "Low", "Close",  "Volume"]
)
x_test, y_test = data.get_test_data(
    3,
    False
)   

print(x_test[startPoint])

4542
realdata             Date   Open   High    Low  Close     Volume       Amount
4542  2019-04-30  13.95  14.22  13.88  14.14  5529309.0   78191416.0
4543  2019-05-06  13.93  13.93  13.10  13.26  9729321.0  132145440.0
[[1.395000e+01 1.422000e+01 1.388000e+01 1.414000e+01 5.529309e+06]
 [1.393000e+01 1.393000e+01 1.310000e+01 1.326000e+01 9.729321e+06]]


In [ ]:
002609捷顺科技
{'column': 'High', '2020-01-13': {'day1': {'mean': 10.339327789896167, 'mid': 10.447180206179619}, 'day2': {'mean': 10.315070630023722, 'mid': 10.61759547472}, 'day3': {'mean': 10.123764358519576, 'mid': 10.526777431964874}}}
{'column': 'Low', '2020-01-13': {'day1': {'mean': 9.5138747605182, 'mid': 9.630221907794475}, 'day2': {'mean': 9.448371759254485, 'mid': 9.460135766491293}, 'day3': {'mean': 9.267974237583113, 'mid': 9.363899476975202}}}


600173卧龙地产
{'column': 'High', '2020-01-13': {'day1': {'mean': 5.220620113519952, 'mid': 5.230991631690412}, 'day2': {'mean': 5.213520888456143, 'mid': 5.209407236874104}, 'day3': {'mean': 5.215852158868685, 'mid': 5.207160414522513}}}
{'column': 'Low', '2020-01-13': {'day1': {'mean': 5.143039583921433, 'mid': 5.136800385918468}, 'day2': {'mean': 5.156178324496606, 'mid': 5.169089575327234}, 'day3': {'mean': 5.168744796165265, 'mid': 5.182775970473886}}}

{'column': 'High', '2020-01-15': {'day1': {'mean': 5.165439975213259, 'mid': 5.168776109218598}, 'day2': {'mean': 5.163824574235827, 'mid': 5.167480159848928}, 'day3': {'mean': 5.175807277050801, 'mid': 5.203082672771997}}}
{'column': 'Low', '2020-01-15': {'day1': {'mean': 5.06853737151809, 'mid': 5.069604581221938}, 'day2': {'mean': 5.085844461421482, 'mid': 5.0918985132128}, 'day3': {'mean': 5.117732808148489, 'mid': 5.110968190599232}}}

{'column': 'High', '2020-01-16': {'day1': {'mean': 5.113705769196153, 'mid': 5.1180160179734235}, 'day2': {'mean': 5.1152046435773375, 'mid': 5.109967353343964}, 'day3': {'mean': 5.133438510045409, 'mid': 5.140734442658722}}}
{'column': 'Low', '2020-01-16': {'day1': {'mean': 5.034414397708141, 'mid': 5.0299963299930095}, 'day2': {'mean': 5.037874845735728, 'mid': 5.0429838576167825}, 'day3': {'mean': 5.05571470523253, 'mid': 5.056901103928685}}}

002330 得利斯
{'column': 'High', '2020-01-13': {'day1': {'mean': 7.863455103036015, 'mid': 7.8481212691403925}, 'day2': {'mean': 7.881653447290882, 'mid': 7.878932193741202}, 'day3': {'mean': 7.926459606273099, 'mid': 7.9366519411280745}}}
{'column': 'Low', '2020-01-13': {'day1': {'mean': 7.6948678264440975, 'mid': 7.686873586103321}, 'day2': {'mean': 7.688281802614219, 'mid': 7.718941535563208}, 'day3': {'mean': 7.702413439498282, 'mid': 7.738762202099896}}}

{'column': 'High', '2020-01-14': {'day1': {'mean': 8.002767618219368, 'mid': 8.006779143121094}, 'day2': {'mean': 7.992868902859743, 'mid': 8.021936438977718}, 'day3': {'mean': 7.976877808128483, 'mid': 8.032098140451126}}}
{'column': 'Low', '2020-01-14': {'day1': {'mean': 7.7552635683719995, 'mid': 7.7564952159672975}, 'day2': {'mean': 7.7536509129568, 'mid': 7.751454677805305}, 'day3': {'mean': 7.750234682170674, 'mid': 7.742528327181936}}}

{'column': 'High', '2020-01-15': {'day1': {'mean': 7.9481674963897095, 'mid': 7.945938410460949}, 'day2': {'mean': 7.940989894334459, 'mid': 7.9476602229941635}, 'day3': {'mean': 7.936431785805617, 'mid': 7.96319680508226}}}
{'column': 'Low', '2020-01-15': {'day1': {'mean': 7.732247904066928, 'mid': 7.731997170117684}, 'day2': {'mean': 7.730306100908666, 'mid': 7.721463676868006}, 'day3': {'mean': 7.724774693765679, 'mid': 7.7080927794426675}}}

600075 新疆天业
{'column': 'High', '2020-01-13': {'day1': {'mean': 5.750655466716736, 'mid': 5.73742640428245}, 'day2': {'mean': 5.689212887441739, 'mid': 5.743211092436686}, 'day3': {'mean': 5.603083374865354, 'mid': 5.648190019056201}}}
{'column': 'Low', '2020-01-13': {'day1': {'mean': 5.496698200725019, 'mid': 5.488259122520685}, 'day2': {'mean': 5.456035109698772, 'mid': 5.451947842352093}, 'day3': {'mean': 5.364975584279746, 'mid': 5.4102749960497025}}}



603998 方盛制药
{'column': 'High', '2020-01-13': {'day1': {'mean': 8.354911968340515, 'mid': 8.355935185824636}, 'day2': {'mean': 8.33000896484044, 'mid': 8.355991509385058}, 'day3': {'mean': 8.304637733893586, 'mid': 8.348202506378295}}}
{'column': 'Low', '2020-01-13': {'day1': {'mean': 8.163344105912373, 'mid': 8.178749185381458}, 'day2': {'mean': 8.148445506231859, 'mid': 8.147460515582935}, 'day3': {'mean': 8.118201903646812, 'mid': 8.087863340806216}}}

{'column': 'High', '2020-01-14': {'day1': {'mean': 8.31664386253804, 'mid': 8.326321915015578}, 'day2': {'mean': 8.293840672680293, 'mid': 8.301692797858268}, 'day3': {'mean': 8.265024733203347, 'mid': 8.253786385897547}}}
{'column': 'Low', '2020-01-13': {'day1': {'mean': 8.150449786975049, 'mid': 8.16657861485146}, 'day2': {'mean': 8.127718433339147, 'mid': 8.113465018896386}, 'day3': {'mean': 8.097815688374453, 'mid': 8.060325473695993}}}

{'column': 'High', '2020-01-15': {'day1': {'mean': 8.279258302239235, 'mid': 8.27904162539635}, 'day2': {'mean': 8.277851701615843, 'mid': 8.284646284137853}, 'day3': {'mean': 8.28718136269669, 'mid': 8.278772598993966}}}
{'column': 'Low', '2020-01-15': {'day1': {'mean': 8.044640782684088, 'mid': 8.056528375446797}, 'day2': {'mean': 8.044298792928458, 'mid': 7.996280844509602}, 'day3': {'mean': 8.052553527237848, 'mid': 7.98344330996275}}}

{'column': 'High', '2020-01-16': {'day1': {'mean': 8.187469031440326, 'mid': 8.086880313530566}, 'day2': {'mean': 8.18383722827281, 'mid': 8.145583908054977}, 'day3': {'mean': 8.197571618085961, 'mid': 8.24936379108578}}}
{'column': 'Low', '2020-01-16': {'day1': {'mean': 7.986585931456181, 'mid': 7.9761676579713825}, 'day2': {'mean': 7.987802391385659, 'mid': 7.978933017849922}, 'day3': {'mean': 7.9977486086301495, 'mid': 7.998892441242933}}}

600273嘉化能源
{'column': 'High', '2020-01-13': {'day1': {'mean': 10.910883505138104, 'mid': 10.874304213495925}, 'day2': {'mean': 10.994302617192268, 'mid': 10.990348282307385}, 'day3': {'mean': 11.084564258004072, 'mid': 11.189982165135442}}}
{'column': 'Low', '2020-01-13': {'day1': {'mean': 10.691568356418982, 'mid': 10.676240629116073}, 'day2': {'mean': 10.710502696000038, 'mid': 10.70697052582167}, 'day3': {'mean': 10.731257395051419, 'mid': 10.696877505518495}}}

{'column': 'High', '2020-01-14': {'day1': {'mean': 11.123907582253217, 'mid': 11.115788582079112}, 'day2': {'mean': 11.125136738164349, 'mid': 11.082420478127897}, 'day3': {'mean': 11.098364255795257, 'mid': 11.035132964216173}}}
{'column': 'Low', '2020-01-14': {'day1': {'mean': 10.799784548070281, 'mid': 10.806725251078605}, 'day2': {'mean': 10.772639211494475, 'mid': 10.699564927741886}, 'day3': {'mean': 10.723452732635662, 'mid': 10.609809467494488}}}

{'column': 'High', '2020-01-15': {'day1': {'mean': 11.01316824213043, 'mid': 11.007201918493957}, 'day2': {'mean': 11.015777126967908, 'mid': 11.052130470704286}, 'day3': {'mean': 11.016971572100186, 'mid': 11.046759607214481}}}
{'column': 'Low', '2020-01-15': {'day1': {'mean': 10.745288274005055, 'mid': 10.746666668280959}, 'day2': {'mean': 10.730064005831256, 'mid': 10.719865852501243}, 'day3': {'mean': 10.706753166927491, 'mid': 10.685701204612851}}}

{'column': 'High', '2020-01-16': {'day1': {'mean': 11.149213262699543, 'mid': 11.16120946487412}, 'day2': {'mean': 11.101850966997443, 'mid': 11.07339643491432}, 'day3': {'mean': 11.035834569152446, 'mid': 11.010115441270171}}}
{'column': 'Low', '2020-01-16': {'day1': {'mean': 10.703354554256425, 'mid': 10.711885266043245}, 'day2': {'mean': 10.713373922341502, 'mid': 10.703751208875328}, 'day3': {'mean': 10.722371608876623, 'mid': 10.695965564539655}}}

{'column': 'High', '2020-01-17': {'day1': {'mean': 11.092546024728566, 'mid': 11.098714287159964}, 'day2': {'mean': 11.067618610302917, 'mid': 11.037269784696399}, 'day3': {'mean': 11.04615998565778, 'mid': 10.997244196394458}}}
{'column': 'Low', '2020-01-17': {'day1': {'mean': 10.784088783726096, 'mid': 10.780402389168739}, 'day2': {'mean': 10.776653205498121, 'mid': 10.750415137019008}, 'day3': {'mean': 10.764232676979153, 'mid': 10.70493250804022}}}

In [1]:
import prediction.predicting as pred

pred.give_me_5_predictions("2020-01-17", '603998', "High")
pred.give_me_5_predictions("2020-01-17", '603998', "Low")

Using TensorFlow backend.


{'column': 'High', '2020-01-17': {'day1': {'mean': 8.118469166553464, 'mid': 8.042929823212326}, 'day2': {'mean': 8.113778415266657, 'mid': 8.068895693756641}, 'day3': {'mean': 8.124018171419273, 'mid': 8.119538252204658}}}
{'column': 'Low', '2020-01-17': {'day1': {'mean': 7.905419918972998, 'mid': 7.9019722822308545}, 'day2': {'mean': 7.91406833243277, 'mid': 7.955619077384472}, 'day3': {'mean': 7.93427318829298, 'mid': 7.8974467083811755}}}


{'column': 'Low',
 '2020-01-17': {'day1': {'mean': 7.905419918972998, 'mid': 7.9019722822308545},
  'day2': {'mean': 7.91406833243277, 'mid': 7.955619077384472},
  'day3': {'mean': 7.93427318829298, 'mid': 7.8974467083811755}}}

In [2]:
import os
import json
import time
import math
import numpy as np

import pandas as pd


stock = "603998"
realdata = pd.read_csv('data/' + stock + '.CSV')

startDate = "2019-09-04"
endDate = "2019-09-18"

selected = realdata.loc[(realdata['Date'] > startDate) & (realdata['Date'] < endDate)]
# print(selected[["Date","High","Low"]])


import ipysheet
sheet = ipysheet.sheet(rows=3, columns=4)
cell1 = ipysheet.cell(0, 0, 'Hello')
cell2 = ipysheet.cell(2, 0, 'World')
cell_value = ipysheet.cell(2,2, 42.)
sheet


Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='Hello'), Cell(colu…

In [ ]:
import os
import json
import time
import math
import numpy as np

import pandas as pd
from pathlib import Path
from ipysheet import from_dataframe, to_dataframe, column, calculation, cell
import ipysheet

from traitlets import link
from ipywidgets import HBox



stock = "000738"
realdata = pd.read_csv('data/' + stock + '.CSV')

startDate = "2019-02-11"
endDate = "2019-02-22"


selected_df = realdata.loc[(realdata['Date'] > startDate) & (realdata['Date'] < endDate)]

# predict_files_to_load 
for index, row in selected_df.iterrows():
#     file_name = 'predictions_result/' + stock + "_" + row['Date'] + ".json"
    file = Path('predictions_result/' + stock + "_" + row['Date'] + ".json")
    if file.is_file():
        with open(file) as json_file:
            data = json.load(json_file)
            '''
            {
                "code":'000738',
                'Date': '2019-02-20',
                "predict_date":'2019-02-21',
                "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
                "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
            }
            '''
#             operation result
            
            '''
            {
                'initial_cash':1000000
                "code":'000738',
                'operation_date': '2019-02-21',
                'p_h':13.44409406979382,
                'p_l':13.44409406979382,
                'r_h':13.10,
                'r_l':12.80,
                'if_sell':false,
                'if_buy':true,
                'buy_amount':3,
                'sell_amount':4,
                'cash_in':0,
                'cash_out':3*13.44,
                'remain_stock':3,
                'remain_cash':87666,
                'total_value':98765
            }
            '''
            
            
            
    else:
        print("file not exist")

In [1]:
d0 = {
    "code":'000738',
    'Date': '2019-02-11',
    "predict_date":'2019-02-12',
    "high":[13.3528405388984822,13.352840538898482,13.352840538898482,13.44409406979382,13.44409406979382],
    "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
}
d1 = {
    "code":'000738',
    'Date': '2019-02-12',
    "predict_date":'2019-02-13',
    "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
    "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
}
d2 = {
    "code":'000738',
    'Date': '2019-02-13',
    "predict_date":'2019-02-14',
    "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
    "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
}
d3 = {
    "code":'000738',
    'Date': '2019-02-14',
    "predict_date":'2019-02-15',
    "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
    "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
}
d4 = {
    "code":'000738',
    'Date': '2019-02-15',
    "predict_date":'2019-02-18',
    "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
    "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
}
d5 = {
    "code":'000738',
    'Date': '2019-02-18',
    "predict_date":'2019-02-19',
    "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
    "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
}
d6 = {
    "code":'000738',
    'Date': '2019-02-19',
    "predict_date":'2019-02-20',
    "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
    "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
}
d7 = {
    "code":'000738',
    'Date': '2019-02-20',
    "predict_date":'2019-02-21',
    "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
    "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
}
d8 = {
    "code":'000738',
    'Date': '2019-02-20',
    "predict_date":'2019-02-21',
    "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
    "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
}
d9 = {
    "code":'000738',
    'Date': '2019-02-20',
    "predict_date":'2019-02-21',
    "high":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382],
    "low":[13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382,13.44409406979382]
}

In [3]:
import os
import json
import time
import math
import numpy as np

import pandas as pd
from pathlib import Path
from ipysheet import from_dataframe, to_dataframe, column, calculation, cell
import ipysheet

from traitlets import link
from ipywidgets import HBox



stock = "000738"
realdata = pd.read_csv('data/' + stock + '.CSV')

startDate = "2019-02-11"
endDate = "2019-02-22"
# realdata['Buy'] = 0
# realdata['Sell'] = 0
realdata['Buy'] = 0
realdata['Sell'] = 0
realdata['buy_price'] = 0
realdata['sell_price'] = 0

selected_df = realdata.loc[(realdata['Date'] > startDate) & (realdata['Date'] < endDate)]

# print(days)

days = [d0, d1, d2, d3, d4, d5, d6, d7, d8]
for data in days:
    selected_df.at[realdata['Date']==data['predict_date'], 'buy_price']=np.mean(data['low'])
    selected_df.at[realdata['Date']==data['predict_date'], 'sell_price']=np.mean(data['high'])

# print(selected_df)   
    
    
    
# shift_df = selected_df.shift(-1)

# for index, row in selected_df.iterrows():
#     file_name = 'predictions_result/' + stock + "_" + row['Date'] + ".json"
#     my_file = Path(file_name)
#     if my_file.is_file(): 
#         with open(file_name) as json_file:
#             data = json.load(json_file)
#             print(data)
#             print()
# #             selected_df.iloc[selected_df['Date']==row['Date']]['next_high'] = data['High']['predictions']['d1']['mean']
# #             selected_df.iloc[selected_df['Date']==row['Date']]['next_low'] = data['Low']['predictions']['d1']['mean']
# #             print(data['Date'])
# #             print(data['High']['predictions']['d1'])
# #             print(data['Low']['predictions']['d1'])
#     else:
#         print(file_name, "file not exist")
#         print()

# selected_df.set_index('Date', inplace=True)



sheet1 = from_dataframe(selected_df[["Date", "High", "Low", "buy_price", 'sell_price', ]])

# sheet = ipysheet.sheet()


# # column2 = ipysheet.column(1, [1.] * 5)

# # sheet
# df2 = pd.DataFrame({'A': 1.,
#                     'B': pd.Timestamp('20130102'),
#                     'C': pd.Series(1, index=list(range(4)), dtype='float32'),
#                     'D': np.array([False, True, False, False], dtype='bool'),
#                     'E': pd.Categorical(["test", "train", "test", "train"]),
#                     'F': 'foo'})

# df2.loc[[0, 2], ['B']] = np.nan

# sheet2 = from_dataframe(df2)
# # sheet2

# sheet = ipysheet.sheet()

# cell1 = ipysheet.cell(0, 0, slider, style={'min-width': '150px'})
# cell3 = ipysheet.cell(2, 2, 42.)
# cell_sum = ipysheet.cell(3, 2, 42.)

# @calculation(inputs=[cell(row=1,column=1), cell(row=1,column=2)], output=cell(row=1,column=3))
# def calculate(a, b):
#     return a + b
# print(cell(row=1,column=1))
sheet1

# HBox((sheet2, sheet1))

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, numeric_format=None, row_end=7, row_start=0, squeez…

In [24]:
import numpy as np
import pandas as pd
from ipysheet import from_dataframe, to_dataframe

dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6, 26), index=dates, columns=list(chr(ord('A') + i) for i in range(26)))

sheet = from_dataframe(df)
sheet

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, row_end=5, row_start=0, squeeze_row=False, type='nu…

In [25]:
df2 = pd.DataFrame({'A': 1.,
                    'B': pd.Timestamp('20130102'),
                    'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                    'D': np.array([False, True, False, False], dtype='bool'),
                    'E': pd.Categorical(["test", "train", "test", "train"]),
                    'F': 'foo'})

df2.loc[[0, 2], ['B']] = np.nan

sheet2 = from_dataframe(df2)
sheet2

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, row_start=0, squeeze_row=False, type='numeric', val…

In [26]:
df3 = to_dataframe(sheet2)
df3

,A,B,C,D,E,F
0,1.0,NaT,1.0,False,test,foo
1,1.0,2013-01-02,1.0,True,train,foo
2,1.0,NaT,1.0,False,test,foo
3,1.0,2013-01-02,1.0,False,train,foo


In [6]:
import numpy as np
from traitlets import link
from ipywidgets import HBox
import bqplot.pyplot as plt
from ipysheet import sheet, cell, column

size = 18
scale = 100.
np.random.seed(0)
x_data = np.arange(size)
y_data = np.cumsum(np.random.randn(size)  * scale)

fig = plt.figure()
axes_options = {'x': {'label': 'Date', 'tick_format': '%m/%d'},
                'y': {'label': 'Price', 'tick_format': '0.0f'}}

scatt = plt.scatter(x_data, y_data, colors=['red'], stroke='black')
fig.layout.width = '70%'

sheet1 = sheet(rows=size, columns=2)
x_column = column(0, x_data)
y_column = column(1, y_data)

link((scatt, 'x'), (x_column, 'value'))
link((scatt, 'y'), (y_column, 'value'))

HBox((fig, sheet1))

In [7]:
import numpy as np
import pandas as pd
from ipysheet import from_dataframe
from ipywidgets import Text, VBox, link

df = pd.DataFrame({'A': 1.,
                   'B': pd.Timestamp('20130102'),
                   'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                   'D': np.array([False, True, False, False], dtype='bool'),
                   'E': pd.Categorical(["test", "train", "test", "train"]),
                   'F': 'foo'})

df.loc[[0, 2], ['B']] = np.nan


s = from_dataframe(df)

search_box = Text(description='Search:')
link((search_box, 'value'), (s, 'search_token'))

VBox((search_box, s))